In [21]:
# 读取csv
import csv
data = []
with open('../Calibration/data/squ2011_88.csv','rt') as f:
    csv_reader = csv.reader(f)
    isHeader = True                 # 判断帧头
    for row in csv_reader:          # 将csv 文件中的数据保存到data中
        if isHeader:
            isHeader = False
            continue
        frame = {
            'time':row[0],
            'xyd':row[1:4],
            'press':row[4],
            'mag4':row[5:17]
            }
        data.append(frame)           # 将字典加入到data数组中
print("read csv file successfully")

read csv file successfully


In [22]:
import numpy as np

length = len(data)
# 获取磁场数据，触点水平位置数据和压力数据
magnetic = np.zeros((length,12))
position = np.zeros((length,2))
for i,frame in enumerate(data):
    magnetic[i] = frame['mag4']
    position[i] = frame['xyd'][:2]
    
print(magnetic)
print(position)

[[ -2.4   -4.5  341.22 ... 108.   111.9  380.91]
 [ -1.2   -3.3  338.8  ... 110.4  111.3  380.91]
 [ -2.7   -3.6  342.67 ... 110.7  111.3  379.94]
 ...
 [  7.5   -5.1  343.64 ... 114.9  107.4  373.16]
 [ 10.2   -6.9  342.67 ... 119.4  105.3  377.52]
 [  6.9   -5.4  340.25 ... 119.1  106.2  375.58]]
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 ...
 [20. 20.]
 [20. 20.]
 [20. 20.]]


In [23]:
# 数据归一化
from sklearn.preprocessing import MinMaxScaler
# 创建 MinMaxScaler 对象
scaler = MinMaxScaler()
# 对触点位置与磁场数据进行 fit_transform 操作
normalized_magnetic = scaler.fit_transform(magnetic)
normalized_position = scaler.fit_transform(position)

In [24]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense

# 划分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(
    normalized_magnetic, normalized_position, test_size = 0.20, random_state = 42)

# 建立 LSTM 模型
model = Sequential()
model.add(LSTM(64, input_shape=(12, 1)))
model.add(Dense(32))
model.add(Dense(2))

# 编译模型
model.compile(loss='mse', 
              optimizer='adam',
              metrics='mae')

# 训练模型
model.fit(
    train_x, 
    train_y, 
    epochs=10, 
    batch_size=64, 
    verbose=2,
    validation_data=(test_x, test_y))

# 测试模型
predict = model.predict(test_x)

# 打印预测结果
print(predict)
print(test_y)

Epoch 1/10
454/454 - 5s - loss: 0.0792 - mae: 0.2305 - val_loss: 0.0549 - val_mae: 0.1912 - 5s/epoch - 11ms/step
Epoch 2/10
454/454 - 2s - loss: 0.0527 - mae: 0.1820 - val_loss: 0.0520 - val_mae: 0.1705 - 2s/epoch - 5ms/step
Epoch 3/10
454/454 - 2s - loss: 0.0352 - mae: 0.1446 - val_loss: 0.0264 - val_mae: 0.1217 - 2s/epoch - 5ms/step
Epoch 4/10
454/454 - 2s - loss: 0.0253 - mae: 0.1225 - val_loss: 0.0230 - val_mae: 0.1171 - 2s/epoch - 5ms/step
Epoch 5/10
454/454 - 2s - loss: 0.0218 - mae: 0.1138 - val_loss: 0.0226 - val_mae: 0.1173 - 2s/epoch - 5ms/step
Epoch 6/10
454/454 - 2s - loss: 0.0188 - mae: 0.1049 - val_loss: 0.0191 - val_mae: 0.1050 - 2s/epoch - 5ms/step
Epoch 7/10
454/454 - 2s - loss: 0.0170 - mae: 0.0994 - val_loss: 0.0148 - val_mae: 0.0927 - 2s/epoch - 5ms/step
Epoch 8/10
454/454 - 2s - loss: 0.0151 - mae: 0.0934 - val_loss: 0.0156 - val_mae: 0.0940 - 2s/epoch - 5ms/step
Epoch 9/10
454/454 - 2s - loss: 0.0143 - mae: 0.0904 - val_loss: 0.0134 - val_mae: 0.0878 - 2s/epoch - 

In [26]:
print(predict[:5,])
print(test_y[:5,])

[[ 0.5027814   0.20450586]
 [ 0.798351    0.06019689]
 [ 0.58414173  0.76149046]
 [ 0.35240614 -0.04938456]
 [ 0.59354377  0.26520303]]
[[0.5 0.2]
 [0.8 0.2]
 [0.5 0.7]
 [0.4 0. ]
 [0.6 0.3]]
